In [2]:
import numpy as np
import matplotlib.pyplot as plt 
import networkx as nx
from IPython.display import display, clear_output
from scipy import signal
from scipy import fftpack
from numpy.fft import fft, fftfreq
import os
import pandas as pd
import time 

# import custom module
import SIRS_twoSF

In [3]:
# needed only if you made some changes at SIRS.py and want to import the updated version of the module
from importlib import reload
reload(SIRS_twoSF)

<module 'SIRS_twoSF' from '/home/clara/OneDrive/Università/Terzo Semestre/Life Data Epidemiology/LDE_project/LifeDataEpidemiology/SIRS_twoSF.py'>

In [4]:
# to install tqdm
#! pip install tqdm

from tqdm import tnrange, trange

# STARTING TO EXPLORE MOBILITY: changing probability of travel p_mob

We fixed the epidemic parameters in a "critical" point at which most of the times the epidemics does not survive and dies out after some iterations. The topology of the two networks is also kept fixed. We now explore the role of the mobility parameter p_mob.

In [10]:
#mobility and topology parameters
N = 1000
I_sf1_init = 5
I_sf2_init = 0
mean_degree = 4
eps = 0.1 

In [11]:
#beta_list = [0.3]
#gamma_list = [0.02]
beta0 = 0.3
gamma = 0.016
mu = 0.15

p_mob_list = np.linspace(0.03,0.2,20)
#p_mob_list_nico = p_mob_list[10:]
#p_mob_list_clara = p_mob_list[:10]
#p_mob_list = p_mob_list[:10]
print('Number of travellers:')
for p in p_mob_list:
    print('p=%.2f    N=%d'%(p, int(p*N)))

Number of travellers:
p=0.03    N=30
p=0.04    N=38
p=0.05    N=47
p=0.06    N=56
p=0.07    N=65
p=0.07    N=74
p=0.08    N=83
p=0.09    N=92
p=0.10    N=101
p=0.11    N=110
p=0.12    N=119
p=0.13    N=128
p=0.14    N=137
p=0.15    N=146
p=0.16    N=155
p=0.16    N=164
p=0.17    N=173
p=0.18    N=182
p=0.19    N=191
p=0.20    N=200


In [ ]:
beta = beta0/mean_degree
infection_params = dict(beta=beta, mu=mu, gamma=gamma)

#R_0 = beta/mu

#print("Basic Reproductive Ratio (taking into account topology):", R_0)
#print("\u03B2 (taking into account topology):", beta)


UP_directory_name = 'data_SF/'+str(beta)+"_"+str(mu)+"_"+str(gamma) 
os.system("mkdir data_SF")
os.system("mkdir "+UP_directory_name)
start = time.time()

for j, p_mob in enumerate(p_mob_list):

    print('Simulation %d/%d'%(j+1,len(p_mob_list)), end =' - ')
    print("P_mob: %.3f"% p_mob)


    #saving stuff for future analysis
    directory_name = UP_directory_name+"/pmob_"+"{:3f}".format(p_mob)
    os.system("mkdir "+directory_name)

    #saving epidemics, topology and mobility parameters

    params=dict(N=N, I_sf1_init=I_sf1_init, I_sf2_nit=I_sf2_init, p_mob=p_mob, mean_degree=mean_degree,
                eps=eps, **infection_params)

    parameters = pd.DataFrame(params, index=[0])
    save=directory_name+"/parameters.csv"
    parameters.to_csv(save)



    #STARTING SIMULATION
    n_runs = 100
    n_iter = 1000


    S_tot_sf2=0
    I_tot_sf2=0
    R_tot_sf2=0
    S_tot_sf1=0
    I_tot_sf1=0
    R_tot_sf1=0


    for run in tnrange(n_runs, desc='1st loop', leave=True):


        # prepare systems
        state_sf1_init, state_sf2_init, variables_net_sf1, variables_net_sf2 = SIRS.prepare_two_sys(N, I_sf1_init, I_sf2_init, 
                                                                                      p_mob, mean_degree)

        #saving initial states
        save = directory_name + "/state_sf1_init_"+str(run)+".txt"
        np.savetxt(save, state_sf1_init.astype(int), fmt ='%d')

        save = directory_name + "/state_sf2_init_"+str(run)+".txt"
        np.savetxt(save, state_sf1_init.astype(int), fmt ='%d')

        save = directory_name + "/travellers_sf1_"+str(run)+".txt"
        travellers_sf1 = variables_net_sf1["travellers_sf1"]
        np.savetxt(save, travellers_sf1.astype(int), fmt ='%d')
        
        save = directory_name + "/travellers_sf2_"+str(run)+".txt"
        travellers_sf2 = variables_net_sf2["travellers_sf2"]
        np.savetxt(save, travellers_sf2.astype(int), fmt ='%d')
        
        #saving initial variables: we just save the IDs of the travelling nodes and their degrees
        save = directory_name + "/deg_trav_sf1_"+str(run)+".txt"
        deg_sf1 = variables_net_sf1["A_sf1"].sum(axis=1)
        deg_trav_sf1 = deg_sf1[travellers_sf1]
        np.savetxt(save, deg_trav_sf1.astype(int), fmt ='%d')
        
        save = directory_name + "/deg_trav_sf2_"+str(run)+".txt"
        deg_sf2 = variables_net_sf2["A_sf2"].sum(axis=1)
        deg_trav_sf2 = deg_sf2[travellers_sf2]
        np.savetxt(save, deg_trav_sf2.astype(int), fmt ='%d')


        
        

        #print("Iteration:", run)

        state_sf1 = np.copy(state_sf1_init)
        state_sf2 = np.copy(state_sf2_init)

        S_sf1 = [N - I_sf1_init]
        I_sf1 = [I_sf1_init]
        R_sf1 = [0]
        S_sf2 = [N - I_sf2_init]
        I_sf2 = [I_sf2_init]
        R_sf2 = [0]
        t_vec = []
        for i in tnrange(n_iter, desc='2nd loop', leave=False):
            state_sf1, state_sf2 = SIRS.two_sys_full_SIRS_step(state_sf1, state_sf2, **variables_net_sf1, 
                                                    **variables_net_sf2, **infection_params)
            S_sf1.append(state_sf1[:,0].sum())
            I_sf1.append(state_sf1[:,1].sum())
            R_sf1.append(state_sf1[:,2].sum())
            S_sf2.append(state_sf2[:,0].sum())
            I_sf2.append(state_sf2[:,1].sum())
            R_sf2.append(state_sf2[:,2].sum())
            #t_vec.append(time.time()-start)

        #tot_time = time.time()-start
        #print("Total time elapsed: %.2f s"%tot_time)
        #print("Time per iteration: %.4f s"%(tot_time/n_iter))

        S_sf1 = np.array(S_sf1)
        I_sf1 = np.array(I_sf1)
        R_sf1 = np.array(R_sf1)
        S_sf2 = np.array(S_sf2)
        I_sf2 = np.array(I_sf2)
        R_sf2 = np.array(R_sf2)
        t_vec = np.array(t_vec)

        #save timeseries of SIR
        SIR_sf1 = np.vstack((S_sf1, I_sf1, R_sf1)).T
        save = directory_name+"/SIR_sf1_"+str(run)+".txt"
        np.savetxt(save, SIR_sf1.astype(int), fmt ='%d')
        SIR_sf2 = np.vstack((S_sf2, I_sf2, R_sf2)).T
        save = directory_name+"/SIR_sf2_"+str(run)+".txt"
        np.savetxt(save, SIR_sf2.astype(int), fmt ='%d')
        
        # broadcast S_tot_sf1 (and all the others) to numpy arrays
        S_tot_sf1+=S_sf1
        I_tot_sf1+=I_sf1
        R_tot_sf1+=R_sf1
        S_tot_sf2+=S_sf2
        I_tot_sf2+=I_sf2
        R_tot_sf2+=R_sf2



    indexes = np.arange(n_iter+1)
    fig, ax = plt.subplots(1,2, figsize=(15,8))
    ax[0].plot(indexes, S_tot_sf1/n_runs, label = r'$S_{sf}$')
    ax[0].plot(indexes, I_tot_sf1/n_runs, label = r'$I_{sf}$')
    ax[0].plot(indexes, R_tot_sf1/n_runs, label = r'$R_{sf}$')
    ax[0].plot(indexes, (S_tot_sf1+I_tot_sf1+R_tot_sf1)/n_runs, label = 'tot')
    ax[0].set_xlabel('Number of iterations', fontsize = 16)
    ax[0].set_ylabel('Number of individuals', fontsize = 16)
    ax[0].set_title("Scale-free network", fontsize = 16)
    ax[0].legend(fontsize=13, loc='upper right')

    ax[1].plot(indexes, S_tot_sf2/n_runs, label = r'$S_{er}$')
    ax[1].plot(indexes, I_tot_sf2/n_runs, label = r'$I_{er}$')
    ax[1].plot(indexes, R_tot_sf2/n_runs, label = r'$R_{er}$')
    ax[1].plot(indexes, (S_tot_sf2+I_tot_sf2+R_tot_sf2)/n_runs, label = 'tot')
    ax[1].set_title("Erdosh-Renyi network", fontsize = 16)
    ax[1].set_xlabel('Number of iterations', fontsize = 16)
    ax[1].set_ylabel('Number of individuals', fontsize = 16)
    ax[1].legend(fontsize=13, loc='upper right')



    plt.suptitle(r"Mean on {} runs, p_mob = {:.3f}; $\beta= {:.3f}; \mu= {:.3f}; \gamma= {:.3f}$".format(n_runs, p_mob, mu, beta, gamma), fontsize=20)
    fig.tight_layout()

    save = directory_name+"/total_"+str(p_mob)+".png"
    fig.savefig(save)
    plt.close()
    
    dt = (time.time() - start)/3600
    # estimated time of arrival
    eta = (dt/(j+1))*(len(p_mob_list)-j-1)
    print("Total time elapsed: %.2f - ETA: %.2f"%(dt, eta))

Simulation 1/20 - P_mob: 0.030
